In [ ]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
import numpy as np
import joblib

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import re

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/ids-dataset.csv')

In [ ]:
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign


In [ ]:
def normalize_col(col):
    col = col.lower()
    col = col.replace('/', '_')
    col = col.replace(' ', '_')
    col = re.sub(r'[^a-z0-9_]', '', col)
    col = re.sub(r'_+', '_', col)
    return col.strip('_')

df.columns = df.columns.map(normalize_col)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   dst_port           1048575 non-null  int64  
 1   protocol           1048575 non-null  int64  
 2   timestamp          1048575 non-null  object 
 3   flow_duration      1048575 non-null  int64  
 4   tot_fwd_pkts       1048575 non-null  int64  
 5   tot_bwd_pkts       1048575 non-null  int64  
 6   totlen_fwd_pkts    1048575 non-null  int64  
 7   totlen_bwd_pkts    1048575 non-null  int64  
 8   fwd_pkt_len_max    1048575 non-null  int64  
 9   fwd_pkt_len_min    1048575 non-null  int64  
 10  fwd_pkt_len_mean   1048575 non-null  float64
 11  fwd_pkt_len_std    1048575 non-null  float64
 12  bwd_pkt_len_max    1048575 non-null  int64  
 13  bwd_pkt_len_min    1048575 non-null  int64  
 14  bwd_pkt_len_mean   1048575 non-null  float64
 15  bwd_pkt_len_std    1048575 non-n

In [ ]:
df.replace([-np.inf,np.inf],np.nan, inplace = True)

In [ ]:
df.describe()

,dst_port,protocol,flow_duration,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,fwd_pkt_len_mean,...,fwd_act_data_pkts,fwd_seg_size_min,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,...,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,4.876262e+03,8.107557e+00,6.255555e+06,6.206622e+00,7.211191e+00,4.479936e+02,4.521803e+03,1.745736e+02,8.389535e+00,3.879579e+01,...,2.793536e+00,2.327970e+01,5.152449e+04,2.136151e+04,8.789157e+04,3.995477e+04,3.101206e+06,7.297218e+05,4.812391e+06,2.126920e+06
std,1.444344e+04,4.460625e+00,1.260291e+09,4.447851e+01,1.048682e+02,1.573541e+04,1.515021e+05,2.876713e+02,1.948279e+01,5.331882e+01,...,5.557106e+00,1.106185e+01,5.815586e+05,2.186405e+05,7.395725e+05,5.602693e+05,5.414780e+08,3.820031e+08,1.522117e+09,1.817013e+07
min,0.000000e+00,0.000000e+00,-9.190110e+11,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.200000e+01,6.000000e+00,7.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.300000e+01,6.000000e+00,1.023000e+03,2.000000e+00,1.000000e+00,3.600000e+01,5.500000e+01,3.400000e+01,0.000000e+00,2.566667e+01,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,6.000000e+00,4.066690e+05,7.000000e+00,6.000000e+00,4.550000e+02,7.680000e+02,1.990000e+02,0.000000e+00,5.550000e+01,...,4.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553300e+04,1.700000e+01,1.200000e+08,5.115000e+03,9.198000e+03,8.591554e+06,1.339773e+07,6.444000e+04,1.460000e+03,1.121703e+04,...,1.031000e+03,4.800000e+01,1.102401e+08,5.723446e+07,1.102401e+08,1.102401e+08,3.394503e+11,2.432682e+11,9.797810e+11,1.260300e+10


In [ ]:
df.dropna(inplace=True)

In [ ]:
df["label"].value_counts()

,count
label,
Benign,663808
FTP-BruteForce,193354
SSH-Bruteforce,187589


In [ ]:
df.replace(to_replace=["FTP-BruteForce","SSH-Bruteforce"],value="Malicious",inplace=True)

In [ ]:
df["label"].value_counts()

,count
label,
Benign,663808
Malicious,380943


In [ ]:
df.replace(to_replace="Benign", value=0, inplace=True)
df.replace(to_replace="Malicious", value=1, inplace =True)

/tmp/ipython-input-3868849377.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace="Malicious", value=1, inplace =True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1044751 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   dst_port           1044751 non-null  int64  
 1   protocol           1044751 non-null  int64  
 2   timestamp          1044751 non-null  object 
 3   flow_duration      1044751 non-null  int64  
 4   tot_fwd_pkts       1044751 non-null  int64  
 5   tot_bwd_pkts       1044751 non-null  int64  
 6   totlen_fwd_pkts    1044751 non-null  int64  
 7   totlen_bwd_pkts    1044751 non-null  int64  
 8   fwd_pkt_len_max    1044751 non-null  int64  
 9   fwd_pkt_len_min    1044751 non-null  int64  
 10  fwd_pkt_len_mean   1044751 non-null  float64
 11  fwd_pkt_len_std    1044751 non-null  float64
 12  bwd_pkt_len_max    1044751 non-null  int64  
 13  bwd_pkt_len_min    1044751 non-null  int64  
 14  bwd_pkt_len_mean   1044751 non-null  float64
 15  bwd_pkt_len_std    1044751 non-null  

In [ ]:
train,test =train_test_split(df,test_size=0.3, random_state=12)

In [ ]:
train_num_cols = train.select_dtypes(include=np.number).columns

In [ ]:
list(train_num_cols)

Index(['dst_port', 'protocol', 'flow_duration', 'tot_fwd_pkts', 'tot_bwd_pkts',
       'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std',
       'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean',
       'bwd_pkt_len_std', 'flow_byts_s', 'flow_pkts_s', 'flow_iat_mean',
       'flow_iat_std', 'flow_iat_max', 'flow_iat_min', 'fwd_iat_tot',
       'fwd_iat_mean', 'fwd_iat_std', 'fwd_iat_max', 'fwd_iat_min',
       'bwd_iat_tot', 'bwd_iat_mean', 'bwd_iat_std', 'bwd_iat_max',
       'bwd_iat_min', 'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags',
       'bwd_urg_flags', 'fwd_header_len', 'bwd_header_len', 'fwd_pkts_s',
       'bwd_pkts_s', 'pkt_len_min', 'pkt_len_max', 'pkt_len_mean',
       'pkt_len_std', 'pkt_len_var', 'fin_flag_cnt', 'syn_flag_cnt',
       'rst_flag_cnt', 'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt',
       'cwe_flag_count', 'ece_flag_cnt', 'down_up_ratio', 'pkt_size_avg',
       'fwd_seg_siz

In [ ]:
scaler = MinMaxScaler()

In [ ]:
train

,dst_port,protocol,timestamp,flow_duration,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,...,fwd_seg_size_min,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
515047,53,17,14/02/2018 11:11:09,11744,1,1,40,99,40,40,...,8,0.0,0.0,0,0,0.0,0.0,0,0,0
546837,3389,6,14/02/2018 03:37:30,2836498,9,7,1144,1581,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
7424,21,6,14/02/2018 10:37:06,1,1,1,0,0,0,0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,1
289213,22,6,14/02/2018 02:47:50,6,1,1,0,0,0,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
126528,21,6,14/02/2018 11:36:55,2,1,1,0,0,0,0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565481,3389,6,14/02/2018 03:43:51,6060626,10,7,1148,1581,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
413144,80,6,14/02/2018 09:09:23,5469566,4,4,97,232,97,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
255371,22,6,14/02/2018 02:31:33,7,1,1,0,0,0,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
761507,3389,6,14/02/2018 01:34:32,2252005,8,7,1128,1581,661,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0


In [ ]:
train.drop(['timestamp'],axis=1, inplace=True)
test.drop(['timestamp'],axis=1, inplace=True)

In [ ]:
X_train, y_train = train.drop(columns=["label"]), np.array(train["label"])
X_test, y_test = train.drop(columns=["label"]), np.array(train["label"])

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [ ]:
joblib.dump(scaler,'scaler.pkl')

['scaler.pkl']

In [ ]:
y_train

array([0, 0, 1, ..., 1, 0, 0])

In [ ]:
model = RandomForestClassifier()

hyperparameters = {
    'n_estimators':[20,10,40],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

In [ ]:
clf = GridSearchCV(
    estimator=model,
    param_grid=hyperparameters,
    cv=5,
    verbose=1,
    n_jobs=-1
)

In [ ]:
clf.fit(X=X_train, y=y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [20, 10, 40]},
             verbose=1)

In [ ]:
print("Best Parameters:", clf.best_params_)
print("Best Estimator:", clf.best_estimator_)

Best Parameters: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 40}
Best Estimator: RandomForestClassifier(n_estimators=40)


In [ ]:
model = clf.best_estimator_

In [ ]:
prediction = model.predict(X_test)
cf_matrix = confusion_matrix(y_test, prediction)

In [ ]:
print(accuracy_score(y_test,prediction))

1.0


In [ ]:
joblib.dump(model,"/content/drive/MyDrive/Datasets/rf_model.joblib")

['/content/drive/MyDrive/Datasets/rf_model.joblib']